# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846811348943                   -0.70    4.5
  2   -7.852317246313       -2.26       -1.53    1.0
  3   -7.852646235757       -3.48       -2.52    3.2
  4   -7.852646677360       -6.35       -3.36    2.2
  5   -7.852646685960       -8.07       -4.73    1.2
  6   -7.852646686724       -9.12       -5.29    3.5
  7   -7.852646686730      -11.27       -6.12    1.5
  8   -7.852646686730      -12.67       -7.63    2.2
  9   -7.852646686730      -14.57       -7.68    2.8
 10   -7.852646686730      -14.75       -8.82    2.0
 11   -7.852646686730   +  -14.45       -9.68    2.5
 12   -7.852646686730      -15.05      -10.83    2.0
 13   -7.852646686730   +  -14.57       -9.23    2.0
 14   -7.852646686730      -14.45       -9.16    1.0
 15   -7.852646686730   +    -Inf       -9.33    1.5
 16   -7.852646686730   +    -Inf       -9.19    1.0
 17   -7.852646686730   +  -15.05       -9.48 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846664795064                   -0.70           4.5
  2   -7.852527149566       -2.23       -1.63   0.80    2.5
  3   -7.852636767246       -3.96       -2.72   0.80    1.0
  4   -7.852646455729       -5.01       -3.28   0.80    2.0
  5   -7.852646682427       -6.64       -4.12   0.80    1.5
  6   -7.852646686485       -8.39       -4.82   0.80    1.5
  7   -7.852646686724       -9.62       -5.69   0.80    2.0
  8   -7.852646686730      -11.26       -6.56   0.80    1.8
  9   -7.852646686730      -12.61       -7.40   0.80    2.2
 10   -7.852646686730      -14.21       -8.22   0.80    2.0
 11   -7.852646686730      -15.05       -8.91   0.80    1.8
 12   -7.852646686730   +  -15.05       -9.60   0.80    1.8
 13   -7.852646686730   +    -Inf      -10.65   0.80    2.2
 14   -7.852646686730      -14.75      -11.19   0.80    2.2
 15   -7.852646686730   +  -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.349417e+01     2.967531e+00
 * time: 0.4081261157989502
     1     1.562719e+00     1.707101e+00
 * time: 0.5993249416351318
     2    -8.690380e-01     2.044256e+00
 * time: 0.6226100921630859
     3    -3.538632e+00     1.768261e+00
 * time: 0.6560511589050293
     4    -4.606056e+00     1.854989e+00
 * time: 0.689439058303833
     5    -6.562587e+00     1.075181e+00
 * time: 0.7228760719299316
     6    -7.353597e+00     4.251942e-01
 * time: 0.7564489841461182
     7    -7.621370e+00     1.622429e-01
 * time: 0.7797400951385498
     8    -7.709872e+00     1.991716e-01
 * time: 0.867401123046875
     9    -7.757860e+00     1.016335e-01
 * time: 0.8910989761352539
    10    -7.785570e+00     8.226029e-02
 * time: 0.9148681163787842
    11    -7.805619e+00     7.546933e-02
 * time: 0.9384541511535645
    12    -7.826198e+00     6.040628e-02
 * time: 0.9622571468353271
    13    -7.842052e+00     4.768254e-02
 * time: 0.985625982284

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846867502874                   -0.70    4.8
  2   -7.852317918083       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686710                   -2.55
  2   -7.852646686730      -10.70       -5.95
  3   -7.852646686730   +  -15.05      -12.73


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.732588288622552e-14
|ρ_newton - ρ_scfv| = 3.0340143542423193e-13
|ρ_newton - ρ_dm|   = 6.028589782203148e-10
